In [ ]:
!pip install BERTopic

In [ ]:
!pip install bertopic
!pip install --upgrade joblib==1.1.0 # HDBScan fails to load with joblib==1.2.0
!pip install --upgrade git+https://github.com/scikit-learn-contrib/hdbscan.git#egg=hdbscan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.5.1
    Uninstalling joblib-1.5.1:
      Successfully uninstalled joblib-1.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-learn 1.6.1 requires joblib>=1.2.0, but you have joblib 1.1.0 which is incompatible.
imbalanced-learn 0.13.0 requires joblib<2,>=1.1.1, but you have joblib 1.1.0 which is incompatible.
  Cloning https://github.com/scikit-learn-contrib/hdbscan.git to /tmp/pip-install-z9x48qsx/hdbscan_21a3d3a13b3848fba016a40598aeafbc
  Running command git clone --filter=blob:none --quiet https://github.com/scikit-learn-contrib/hdbscan.git /tmp/pip-install-z9x48qsx/hdbscan_21a3d3a13b3848fba016a40598aeafbc
  Resolved https://github.com/scikit-learn-contrib/hdbscan.git to commit 8ecf239379c75343b8e135

In [ ]:

import os
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP # In case we want to make results deterministic and reproducible

In [ ]:
with open("stop-words-russian.txt", 'r', encoding="utf8") as f:
    stop = f.read().splitlines()

vectorizer_model = CountVectorizer(ngram_range=(1, 1), stop_words=frozenset(stop))

In [ ]:
# Prepare data
df = pd.read_csv('processed_dataset.csv', encoding="utf-8")
docs = df['text_ready'].values.tolist()


In [ ]:
df.head()

,Unnamed: 0,text,sentiment,text_lower,ready,text_ready
0,43956,Развода на деньги нет\nНаблюдаюсь в Лайфклиник...,1,р а з в о д д е н ь г а н а б л ю д а т ь ...,"['развода', 'деньги', 'наблюдаюсь', 'лайфклини...",развод деньга наблюдаться лайфклиника беременн...
1,17755,Отель выбрали потому что рядом со стадионом. О...,0,в ы б р а т ь р я д о м с т а д и о н н ...,"['выбрали', 'рядом', 'стадионом', '4', 'номер'...",выбрать рядом стадион номер большой кровать сп...
2,20269,"Вылечили\nГноился с рождения глазик, в поликли...",1,в ы л е ч и т ь г н о и т ь с я р о ж д е ...,"['вылечили', 'гноился', 'рождения', 'глазик', ...",вылечить гноиться рождение глазик поликлиника ...
3,16648,Хорошее расположение.С вокзала дошли пешком.Но...,0,х о р о ш и й р а с п о л о ж е н и е в о ...,"['хорошее', 'расположение', 'с', 'вокзала', 'д...",хороший расположение вокзал дойти пешком ночью...
4,27879,"Отличное месторасположение,прекрасный вид,особ...",1,о т л и ч н ы й м е с т о р а с п о л о ж е ...,"['отличное', 'месторасположение', 'прекрасный'...",отличный месторасположение прекрасный вид особ...


In [ ]:
from bertopic import BERTopic
import pandas as pd
import nltk
nltk.download('stopwords')
#from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df.head()

,Unnamed: 0,text,sentiment,text_lower,ready,text_ready
0,43956,Развода на деньги нет\nНаблюдаюсь в Лайфклиник...,1,р а з в о д д е н ь г а н а б л ю д а т ь ...,"['развода', 'деньги', 'наблюдаюсь', 'лайфклини...",развод деньга наблюдаться лайфклиника беременн...
1,17755,Отель выбрали потому что рядом со стадионом. О...,0,в ы б р а т ь р я д о м с т а д и о н н ...,"['выбрали', 'рядом', 'стадионом', '4', 'номер'...",выбрать рядом стадион номер большой кровать сп...
2,20269,"Вылечили\nГноился с рождения глазик, в поликли...",1,в ы л е ч и т ь г н о и т ь с я р о ж д е ...,"['вылечили', 'гноился', 'рождения', 'глазик', ...",вылечить гноиться рождение глазик поликлиника ...
3,16648,Хорошее расположение.С вокзала дошли пешком.Но...,0,х о р о ш и й р а с п о л о ж е н и е в о ...,"['хорошее', 'расположение', 'с', 'вокзала', 'д...",хороший расположение вокзал дойти пешком ночью...
4,27879,"Отличное месторасположение,прекрасный вид,особ...",1,о т л и ч н ы й м е с т о р а с п о л о ж е ...,"['отличное', 'месторасположение', 'прекрасный'...",отличный месторасположение прекрасный вид особ...


In [ ]:
docs

['развод деньга наблюдаться лайфклиника беременность развод деньга врач плохой повод ресепшен согласиться путать документ',
 'выбрать рядом стадион номер большой кровать спальная одеяло подушка зона отдых чайный сет бутылка бесплатно каждый день шкаф минь бар утюг гладильный доска отдельно прибор глажка брюки санузел общий большой ванна лейка съёмный раковина фен халат тапочки расширить пакет косметический средство соль ванна интернет быстрый этаж высокий вид окно город недостаток номер 919 напротив хозлифт утро начинать пользоваться персонал обслуживание завтрак этаж слышный звон посуда разговор завтрак включить стоить 23 евро рядом отель кафе subway',
 'вылечить гноиться рождение глазик поликлиника назначить лечение помочь сходить сюда вылечить офтальмолог бобров понравиться рассказать показать несколько правильно делать также приходить массажистка ольга внимательный профессионал свой дело спасибо',
 'хороший расположение вокзал дойти пешком ночью тихо хотя центр рядом наш номер башн

In [ ]:
from nltk.corpus import stopwords


In [ ]:
# Remove stopwords **after** generating the clusters
# NOTE: You can also perform the lemmatization here
my_stopwords = stopwords.words("russian")
vectorizer_model = CountVectorizer(stop_words=my_stopwords)

In [ ]:
#from hdbscan import HDBSCAN
documents = df['text_ready'].values.tolist()
documents_limited = documents[:15000]
umap_model = UMAP(n_neighbors=10, n_components=3,
                  min_dist=0.0, metric='cosine', random_state=13)

In [ ]:
# Load KBLab's Swedish sentence transformer model
sentence_model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2", device="cpu")

# Initialize BERTopic with the settings we want
topic_model = BERTopic(embedding_model=sentence_model,
                       vectorizer_model=vectorizer_model,
                       umap_model=umap_model,
                       calculate_probabilities=True,
                       verbose=True)

# Fit the model
#topics, probs = topic_model.fit_transform(documents)

In [ ]:
# Fit the model
topics, probs = topic_model.fit_transform(documents_limited)

2025-06-15 16:57:14,202 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/469 [00:00<?, ?it/s]

2025-06-15 17:59:04,106 - BERTopic - Embedding - Completed ✓
2025-06-15 17:59:04,112 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-06-15 17:59:16,492 - BERTopic - Dimensionality - Completed ✓
2025-06-15 17:59:16,494 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-06-15 17:59:22,581 - BERTopic - Cluster - Completed ✓
2025-06-15 17:59:22,598 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-06-15 17:59:23,859 - BERTopic - Representation - Completed ✓


In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1595,-1_год_тенг_суд_казахстан,"[год, тенг, суд, казахстан, дело, договор, бан...",[бисекей хотеть изменить экономика мешать бизн...
1,0,4104,0_врач_отделение_ребёнок_клиника,"[врач, отделение, ребёнок, клиника, больница, ...",[огромный благодарность хирургический отделени...
2,1,3750,1_номер_отель_завтрак_хороший,"[номер, отель, завтрак, хороший, персонал, ряд...",[отдыхать отель 28 ноябрь 21 декабрь 2017 год ...
3,2,2202,2_ресторан_кухня_официант_блюдо,"[ресторан, кухня, официант, блюдо, меню, интер...",[сегодня собраться отметить день рождение наш ...
4,3,613,3_зуб_клиника_стоматолог_врач,"[зуб, клиника, стоматолог, врач, стоматология,...",[сложный удаление зуб клиника пойти исключител...
...,...,...,...,...,...
63,62,12,62_наркоман_наркотик_алкоголь_сажать,"[наркоман, наркотик, алкоголь, сажать, продажа...",[коллега негосударственный помогать организаци...
64,63,10,63_служить_труд_обвинить_мигрант,"[служить, труд, обвинить, мигрант, продавать, ...",[сочетание дикий условие жизнь человек образов...
65,64,10,64_воспроизведение_авторизация_произведение_ли...,"[воспроизведение, авторизация, произведение, л...",[авторизация сайт выдавать огромный архив связ...
66,65,10,65_фарватер_азовский_водоизмещение_корабль,"[фарватер, азовский, водоизмещение, корабль, л...",[казать точный число каждый проект часто встре...


In [ ]:
topic_model.get_topic(0)

[('врач', np.float64(0.02783040489011447)),
 ('отделение', np.float64(0.014530709282753058)),
 ('ребёнок', np.float64(0.013980539778722652)),
 ('клиника', np.float64(0.012557513215276709)),
 ('больница', np.float64(0.011937803125271738)),
 ('благодарность', np.float64(0.011578878411230741)),
 ('доктор', np.float64(0.010370278311373205)),
 ('сказать', np.float64(0.01036109238360931)),
 ('хотеть', np.float64(0.01017761411542134)),
 ('анализ', np.float64(0.010033542407352741))]

In [ ]:
topic_model.get_representative_docs(0)

['огромный благодарность хирургический отделение хотеться отзыв затеряться поток благодарность написать хороший правильный обезличиваться постоянный употребление слово который готовый повторить высокий профессионализм внимательность чуткость ответственность отзывчивость причём превосходный степень применительно весь работать хирургический отделение весь особенность зав отеделенить мачерет евгений александрович лечить врач захаров антон николаевич мамонтов роман евгениевич весь работать хотеться выразить благодарность услышать заметить отметить поэтому написать переложить слово мысль ощущение который возникать находиться лечение полный сказать олимпийский спокойствие безусловный доверие работа врач медперсонал операционный стол больничный койка свой опыт административный деятельность тесный контакт медицина течение 35 год видеть чувствовать профессионализм отношение врач персонал работа пациент уверенность голословный плохо отойти день ночью сделать возможный невозможный стать лёгкий ле

In [ ]:
topic_model.get_topic(4) # topic_model.get_topic(similar_topics[0])

[('машина', np.float64(0.05201979546600158)),
 ('автомобиль', np.float64(0.034448721570903854)),
 ('салон', np.float64(0.023337377967805342)),
 ('авто', np.float64(0.020596987890346086)),
 ('двигатель', np.float64(0.020411010468199178)),
 ('задний', np.float64(0.01695435988398588)),
 ('сидение', np.float64(0.016854509589379275)),
 ('багажник', np.float64(0.01586736530665837)),
 ('км', np.float64(0.01582295312793147)),
 ('ездить', np.float64(0.014871663988239135))]

In [ ]:
topic_model.visualize_topics()

In [ ]:
fig = topic_model.visualize_topics()
fig.write_html("/content/sample_data/ria_topic_20t.html")

In [ ]:
topic_model.visualize_hierarchy()

In [ ]:
hierarchical_topics = topic_model.hierarchical_topics(documents_limited)
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

100%|██████████| 66/66 [00:00<00:00, 219.46it/s]


In [ ]:
topic_model.reduce_topics(docs, nr_topics=20)


ValueError: All arrays must be of the same length

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1595,-1_год_тенг_суд_казахстан,"[год, тенг, суд, казахстан, дело, договор, бан...",[бисекей хотеть изменить экономика мешать бизн...
1,0,4104,0_врач_отделение_ребёнок_клиника,"[врач, отделение, ребёнок, клиника, больница, ...",[огромный благодарность хирургический отделени...
2,1,3750,1_номер_отель_завтрак_хороший,"[номер, отель, завтрак, хороший, персонал, ряд...",[отдыхать отель 28 ноябрь 21 декабрь 2017 год ...
3,2,2202,2_ресторан_кухня_официант_блюдо,"[ресторан, кухня, официант, блюдо, меню, интер...",[сегодня собраться отметить день рождение наш ...
4,3,613,3_зуб_клиника_стоматолог_врач,"[зуб, клиника, стоматолог, врач, стоматология,...",[сложный удаление зуб клиника пойти исключител...
...,...,...,...,...,...
63,62,12,62_наркоман_наркотик_алкоголь_сажать,"[наркоман, наркотик, алкоголь, сажать, продажа...",[коллега негосударственный помогать организаци...
64,63,10,63_служить_труд_обвинить_мигрант,"[служить, труд, обвинить, мигрант, продавать, ...",[сочетание дикий условие жизнь человек образов...
65,64,10,64_воспроизведение_авторизация_произведение_ли...,"[воспроизведение, авторизация, произведение, л...",[авторизация сайт выдавать огромный архив связ...
66,65,10,65_фарватер_азовский_водоизмещение_корабль,"[фарватер, азовский, водоизмещение, корабль, л...",[казать точный число каждый проект часто встре...


In [ ]:
topic_model.get_topic(3)

[('зуб', np.float64(0.08486854686843683)),
 ('клиника', np.float64(0.038493568649341656)),
 ('стоматолог', np.float64(0.029541019235078927)),
 ('врач', np.float64(0.025852433638421157)),
 ('стоматология', np.float64(0.025296357083869007)),
 ('пломба', np.float64(0.021906354859310872)),
 ('лечение', np.float64(0.020762365350552224)),
 ('доктор', np.float64(0.01941318637470407)),
 ('лечить', np.float64(0.018212100964917145)),
 ('коронка', np.float64(0.016261506307574964))]

In [ ]:
topic_model.visualize_topics()


In [ ]:
topic_model.visualize_barchart(width=280, height=330,top_n_topics=20, n_words=20)


In [ ]:
fig4.write_html("/content/sample_data/ria_topic_20wordBars.html")

NameError: name 'fig4' is not defined

In [ ]:
fig5 = topic_model.visualize_heatmap(n_clusters=9)
fig5.write_html("/content/sample_data/ria_topic_20corr.html")